# 如何清理資料
https://www.perplexity.ai/search/cc086321-725e-4231-a50a-8d53131ab4e2

In [ ]:
import pandas as pd

df = pd.read_csv('各鄉鎮市區人口密度.csv') 
#讀取csv檔，此採取相對路徑，須放在同一個資料夾。
df

In [ ]:
df.info() #查詢欄位資訊
df1 = df.rename(columns={'statistic_yyy':'統計年',
                        'site_id':'區域別',
                        'people_total':'年底人口數',
                        'area':'土地面積',
                        'population_density':'人口密度'
                        }) 
                        
                        #變更欄位名稱

df1

In [ ]:
df2 = df1.drop( index=0 ) # 刪除第0列
df3 = df2.dropna ( axis =0 , how = 'any'  ) #清理帶有na的資料
df3

In [ ]:
df4 = df3.drop('統計年',axis=1) #第一項為引數值呼叫，後面為引數名稱呼叫。
df4.info()
df4

In [ ]:
def strToInt (x:str)->int: #定義函式:字串轉為數值。
    try:
        return int(x)
        #更改資料類型，字串變成數值int。
    except:
        return 0
        #遇到錯誤(非int)，則回傳0。

df4['年底人口數'] = df4['年底人口數'].apply(strToInt) #呼叫函式。用新生成的數值，取代原本的資料。
df4['人口密度'] = df4['人口密度'].apply(strToInt)

df4['土地面積'] = df4['土地面積'].astype('float') #str轉成浮點數。

df4.info()
df4

#至此資料清理完成，可以進行後續製作遮罩、運算(比較)，以進行搜尋。


In [ ]:
#把區域別欄位，縣市鄉鎮，拆成兩欄。

def split_area(v:str)->str:
    if len(v) >= 5:
        return v[:3]
    else:
        return v

def split_area1(v:str)->str:
    if len(v) == 6:
        return v [-3:]
    elif len(v) == 5:
        return v [-2:]
    else:
        return v

df4['縣市'] = df4['區域別'].apply(split_area)
df4['區域'] = df4['區域別'].apply(split_area1)

df4




In [ ]:
df5 = df4.reindex(columns=['縣市','區域','年底人口數','土地面積','人口密度'])
df5

In [ ]:
#搜尋
df5.query('年底人口數>=500000')

In [ ]:
m = df5['人口密度'].max()
df5.query('人口密度==@m')

In [48]:
# 分組彙總
df6 = df5.groupby(by='縣市')[['年底人口數','土地面積','人口密度']].agg([('平均','mean'),('加總','sum')])

#在分組完成後，這個部分會從每一個分組中，選取 年底人口數、土地面積、人口密度 這三個欄位。接下來的計算將只針對這三個欄位進行。
#.agg(): 是 aggregate 的縮寫，意思是「彙總」。它會對前面選定的三個欄位進行指定的數學運算。

'''
此行程式碼的意思：將 df5 這個資料表，按照「縣市」進行分組。
針對每個縣市分組，挑出「年底人口數」、「土地面積」、「人口密度」這三項數據，
並分別計算它們的「平均值」和「總和」。
最後將這個彙總後的結果，存成一個新的表格 df6。
'''
df6

年底人口數                 土地面積                     人口密度        
                 平均       加總          平均         加總            平均      加總
縣市                                                                       
南投縣    38542.384615   501051  315.879692  4106.4360    338.692308    4403
南沙群島       0.000000        0    0.504500     0.5045      0.000000       0
嘉義市   134699.000000   269398   30.012800    60.0256   4490.000000    8980
嘉義縣    28399.000000   511182  105.757594  1903.6367    383.666667    6906
基隆市    53065.428571   371458   18.965557   132.7589   4603.857143   32227
宜蘭縣    38050.583333   456607  178.635425  2143.6251   1125.416667   13505
屏東縣    25149.666667   829939   84.109100  2775.6003    567.666667   18733
彰化縣    49325.307692  1282458   41.322923  1074.3960   1293.307692   33626
新北市   147246.296296  3975650   61.485107  1660.0979   7190.629630  194147
新竹市   147044.000000   441132   34.717533   104.1526   5776.333333   17329
新竹縣    45552.583333   546631  115.061325  1380.7359    839.750000   10077
東沙群島       0.000000        0    2.380000     2.3800      0.000000       0
桃園市   148566.750000  1782801   95.369500  1144.4340   2863.000000   34356
澎湖縣    17345.500000   104073   21.144017   126.8641    689.166667    4135
臺中市    96105.862069  2787070   76.375752  2214.8968   4997.103448  144916
臺北市   223604.750000  2683257   22.649975   271.7997  15377.666667  184532
臺南市    50987.081081  1886522   59.233868  2191.6531   2006.864865   74254
臺東縣    13721.250000   219540  219.703287  3515.2526    133.375000    2134
花蓮縣    25325.923077   329237  356.043954  4628.5714    466.692308    6067
苗栗縣    30767.055556   553807  101.128606  1820.3149    571.222222   10282
連江縣     4952.000000     9904   10.150000    20.3000    481.500000     963
金門縣    22909.333333   137456   25.276000   151.6560    932.666667    5596
雲林縣    34518.650000   690373   64.541630  1290.8326    560.050000   11201
高雄市    73076.631579  2776912   77.680326  2951.8524   5116.052632  194410

In [ ]:
df6.columns.names=['資料','統計']
df6

In [ ]:
#資料重塑（Reshaping）
#此行程式碼的意思：將一個以「縣市」為主要列（Row）的報表，重塑成一個以「統計方法」（平均、加總）為主要列的新報表。

df6.stack(level=['資料','統計'],future_stack=True).unstack(level=['縣市','資料'])

'''
第一步：.stack(level=['資料','統計'], future_stack=True)
stack(): 這個函式會將**欄位（Column）的層級"堆疊"到索引（Index）**中，讓表格變得更「長」、更「窄」。
level=['資料','統計']: 這表示我們要將名為 資料（'年底人口數', '土地面積', ...）和 統計（'平均', '加總'）的這兩層欄位全部堆疊到索引裡。
future_stack=True: 這是一個參數，用來確保 stack 的行為符合 pandas 新版本的預期，讓堆疊的層級總是加在索引的最內層。

第二步：.unstack(level=['縣市','資料'])
unstack(): 這是 stack() 的反向操作。它會將**索引（Index）的層級"攤開"到欄位（Column）**中，讓表格變得更「寬」。
level=['縣市','資料']: 這表示我們要從剛剛產生的三層索引中，挑出 縣市 和 資料 這兩層，把它們攤開變成新的欄位。
'''

In [50]:
# 建議的寫法

# 0. (前提) 為欄位層級命名，增加程式碼可讀性
df6.columns.names = ['資料', '統計']

# 1. 將欄位堆疊到索引中，形成一個長格式的 Series
# "stack" the columns into a long-format Series
df_long = df6.stack(level=['資料', '統計'], future_stack=True)

# 2. 將部分索引攤開成新的欄位，完成重塑
# "unstack" the desired levels to form the new columns
df_reshaped = df_long.unstack(level=['縣市', '資料'])

# df_reshaped 現在就是你想要的最終結果
df_reshaped


縣市            南投縣                            南沙群島                    嘉義市  \
資料          年底人口數         土地面積         人口密度 年底人口數    土地面積 人口密度     年底人口數   
統計                                                                         
平均   38542.384615   315.879692   338.692308   0.0  0.5045  0.0  134699.0   
加總  501051.000000  4106.436000  4403.000000   0.0  0.5045  0.0  269398.0   

縣市                        嘉義縣  ...    連江縣            金門縣           \
資料     土地面積    人口密度     年底人口數  ...   人口密度          年底人口數     土地面積   
統計                             ...                                  
平均  30.0128  4490.0   28399.0  ...  481.5   22909.333333   25.276   
加總  60.0256  8980.0  511182.0  ...  963.0  137456.000000  151.656   

縣市                     雲林縣                                 高雄市               \
資料         人口密度      年底人口數        土地面積      人口密度         年底人口數         土地面積   
統計                                                                            
平均   932.666667   34518.65    64.54163    560.05  7.307663e+04    77.680326   
加總  5596.000000  690373.00  1290.83260  11201.00  2.776912e+06  2951.852400   

縣市                 
資料           人口密度  
統計                 
平均    5116.052632  
加總  194410.000000  

[2 rows x 72 columns]